### Generating example restaurant Data

In [4]:
import csv
import random

# Function to generate random reviews
def generate_reviews():
    positive = ["Delicious!", "Great taste!", "Highly recommended!", "Amazing flavor!"]
    negative = ["Disappointing", "Not worth it", "Could be better", "Mediocre"]
    return ", ".join(random.choices(positive + negative, k=random.randint(1, 3)))

# Generate restaurant data
items = [
    "Margherita Pizza", "Pepperoni Pizza", "Vegetarian Pizza", "Hawaiian Pizza",
    "Spaghetti Carbonara", "Fettuccine Alfredo", "Lasagna", "Chicken Parmesan",
    "Caesar Salad", "Greek Salad", "Tiramisu", "Chocolate Mousse"
]

with open('restaurant_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Item", "Description", "Price", "Availability", "Reviews"])
    
    for item in items:
        description = f"Delicious {item.lower()} made with fresh ingredients"
        price = round(random.uniform(8.99, 24.99), 2)
        availability = random.choice(["In Stock", "Low Stock", "Out of Stock"])
        reviews = generate_reviews()
        
        writer.writerow([item, description, price, availability, reviews])

print("Restaurant data CSV file has been generated.")

Restaurant data CSV file has been generated.


### Order Placement Function

In [5]:
import json
from datetime import datetime

def place_order(items, customer_name, customer_address):
    order = {
        "order_id": f"ORD-{datetime.now().strftime('%Y%m%d%H%M%S')}",
        "customer_name": customer_name,
        "customer_address": customer_address,
        "items": items,
        "total_amount": sum(item["price"] * item["quantity"] for item in items),
        "order_date": datetime.now().isoformat(),
        "status": "Placed"
    }
    return json.dumps(order, indent=2)

# Example usage
sample_order = place_order(
    items=[
        {"name": "Margherita Pizza", "price": 12.99, "quantity": 2},
        {"name": "Caesar Salad", "price": 8.99, "quantity": 1}
    ],
    customer_name="John Doe",
    customer_address="123 Main St, Anytown, USA"
)

print(sample_order)

{
  "order_id": "ORD-20240908174636",
  "customer_name": "John Doe",
  "customer_address": "123 Main St, Anytown, USA",
  "items": [
    {
      "name": "Margherita Pizza",
      "price": 12.99,
      "quantity": 2
    },
    {
      "name": "Caesar Salad",
      "price": 8.99,
      "quantity": 1
    }
  ],
  "total_amount": 34.97,
  "order_date": "2024-09-08T17:46:36.544058",
  "status": "Placed"
}


### Vector Database Setup

In [6]:
import csv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# Load restaurant data
def load_restaurant_data(file_path='restaurant_data.csv'):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        return list(reader)

# Create documents from restaurant data
def create_documents(data):
    documents = []
    for item in data:
        content = f"Item: {item['Item']}\nDescription: {item['Description']}\nPrice: ${item['Price']}\nAvailability: {item['Availability']}\nReviews: {item['Reviews']}"
        documents.append(Document(page_content=content, metadata={"item": item['Item']}))
    return documents

# Set up vector database
def setup_vector_db():
    # Load data
    restaurant_data = load_restaurant_data()
    documents = create_documents(restaurant_data)

    # Initialize HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    # Create Chroma vector store
    vectorstore = Chroma.from_documents(documents, embeddings, collection_name="restaurant_info")

    return vectorstore

# Run this function to set up the vector database
vectorstore = setup_vector_db()
print("Vector database has been set up successfully.")

/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/shaonsikder/.pyenv/versions/3.11.0/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
I0000 00:00:1725796178.390779 10816432 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Vector database has been set up successfully.


### Restaurant Bot

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import Runnable
from langchain.schema.runnable.config import RunnableConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key="YOUR_GOOGLE_API_KEY",
)

# Set up memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Set up retrieval chain with memory
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful restaurant chatbot. Use the provided context and chat history to answer customer queries about menu items, prices, availability, and reviews. You can also place orders for customers.
    
    Context: {context}
    Chat History: {chat_history}
    
    Remember to be polite and helpful. If you need to place an order, use the place_order function."""),
    ("human", "{input}")
])

# Create the chain
chain = prompt | llm | StrOutputParser()

# Function to run the chatbot
def run_chatbot(user_input: str) -> str:
    # Retrieve relevant information from the vector database and get response
    result = qa_chain({"question": user_input})
    context = result['answer']
    chat_history = memory.chat_memory.messages

    # Generate final response using the retrieved context and chat history
    response = chain.invoke({
        "input": user_input,
        "context": context,
        "chat_history": chat_history
    })
    return response

# Main interaction loop
print("Welcome to our restaurant chatbot! How can I assist you today?")
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        print("Thank you for using our chatbot. Goodbye!")
        break
    
    response = run_chatbot(user_input)
    print("Chatbot:", response)

I0000 00:00:1725796537.975955 10816432 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1725796537.976196 10816432 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Welcome to our restaurant chatbot! How can I assist you today?
Chatbot: The best reviewed item is the Lasagna. 

Chatbot: Yes, the Lasagna is available now. Would you like to place an order for it? 

Chatbot: The price of the Lasagna is $16.37. 

Chatbot: I can place an order for the Lasagna for you. Could you please tell me your name and phone number so I can place the order? 

Chatbot: Please provide me with your delivery address so I can place your order for the Lasagna. 

Chatbot: I'm sorry, I don't have any information about table numbers.  I can help you place an order for the Lasagna if you'd like.  Do you have a delivery address? 

Chatbot: I understand you'd like to place an order for the Lasagna. To complete your order, I need a delivery address or table number. Please provide me with that information. 

Chatbot: I'm sorry, but I don't have any information about a delivery address. I can help you with information about our menu items, prices, and availability.  Do you have an